In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local') \
        .appName('Quiz2') \
        .config('spark.ui.port', '4050') \
        .getOrCreate()

sc = spark.sparkContext

문제 1.

3년 연속 같은 상을 수상한 선수를 찾아 그 3년간의 연봉평균을 출력하시오
만일, 3년 초과 같은 상을 수상하였다면 모든 연속된 3년에 대해 연봉평균을 출력하시오

In [58]:
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import pandas_udf

# playerID,awardID,yearID,lgID,tie,notes
# yearID,teamID,lgID,playerID,salary

awards_player_schema = StructType([
    StructField("playerID", StringType(), False),
    StructField("awardID", StringType(), False),
    StructField("yearID", LongType(), False),
    StructField("lgID", StringType(), False),
    StructField("tie", StringType(), True),
    StructField("notes", StringType(), True),
])

awards_player = spark.read.format("csv").option('header', 'false').option("sep", ",").schema(awards_player_schema).load("../../data/input4/AwardsPlayers.csv")

salaries_schema = StructType([
    StructField("yearID", LongType(), False),
    StructField("teamID", StringType(), False),
    StructField("lgID", StringType(), False),
    StructField("playerID", StringType(), False),
    StructField("salary", LongType(), False),
])

salaries = spark.read.format("csv").option('header', 'false').option("sep", ",").schema(salaries_schema).load("../../data/input4/Salaries.csv")

In [34]:
awards_player.show(5)
salaries.show(5)

+---------+--------+------+----+----+-----+
| playerID| awardID|yearID|lgID| tie|notes|
+---------+--------+------+----+----+-----+
|whitefr01|ALCS MVP|  1980|  AL|null| null|
|nettlgr01|ALCS MVP|  1981|  AL|null| null|
| lynnfr01|ALCS MVP|  1982|  AL|null| null|
|boddimi01|ALCS MVP|  1983|  AL|null| null|
|gibsoki01|ALCS MVP|  1984|  AL|null| null|
+---------+--------+------+----+----+-----+
only showing top 5 rows

+------+------+----+---------+-------+
|yearID|teamID|lgID| playerID| salary|
+------+------+----+---------+-------+
|  1985|   BAL|  AL|murraed02|1472819|
|  1985|   BAL|  AL| lynnfr01|1090000|
|  1985|   BAL|  AL|ripkeca01| 800000|
|  1985|   BAL|  AL| lacyle01| 725000|
|  1985|   BAL|  AL|flanami01| 641667|
+------+------+----+---------+-------+
only showing top 5 rows



In [47]:
cond = [awards_player['playerID'] == salaries['playerID'], awards_player['yearID'] == salaries['yearID']]
awards_salaries = awards_player.join(salaries, cond, 'inner').select(awards_player['playerID'], 'awardID', awards_player['yearID'], 'salary')
awards_salaries.show(5)

+---------+--------------+------+-------+
| playerID|       awardID|yearID| salary|
+---------+--------------+------+-------+
|ripkeca01|  TSN All-Star|  1985| 800000|
|ripkeca01|Silver Slugger|  1985| 800000|
|boggswa01|  TSN All-Star|  1985|1000000|
|evansdw01|    Gold Glove|  1985| 800000|
|pettiga01|    Gold Glove|  1985| 125000|
+---------+--------------+------+-------+
only showing top 5 rows



In [45]:
gap = awards_salaries.groupBy('playerID', 'awardID').count().where('count>=3')
gap.show(5)

+---------+--------------+-----+
| playerID|       awardID|count|
+---------+--------------+-----+
|fernato01|    Gold Glove|    4|
|ordonma01|  TSN All-Star|    3|
|molitpa01|Silver Slugger|    4|
|martied01|  TSN All-Star|    4|
|chaveer01|    Gold Glove|    6|
+---------+--------------+-----+
only showing top 5 rows



In [77]:
cond = [awards_salaries['playerID'] == gap['playerID'], awards_salaries['awardID'] == gap['awardID']]
awards_salaries3 = awards_salaries.join(gap, cond, 'inner').select(awards_salaries['playerID'], awards_salaries['awardID'], 'yearID', 'salary')
awards_salaries3 = awards_salaries3.sort('playerID', 'awardID', 'yearID')
awards_salaries3.show(5)

+---------+----------+------+-------+
| playerID|   awardID|yearID| salary|
+---------+----------+------+-------+
|alomaro01|Gold Glove|  1991|1300000|
|alomaro01|Gold Glove|  1992|2983333|
|alomaro01|Gold Glove|  1993|4933333|
|alomaro01|Gold Glove|  1994|5433334|
|alomaro01|Gold Glove|  1995|5500000|
+---------+----------+------+-------+
only showing top 5 rows



In [82]:
from pyspark.sql.functions import lit,row_number,col
from pyspark.sql.window import Window

w = Window().partitionBy(lit('a')).orderBy(lit('a'))
awards_salaries3_row = awards_salaries3.withColumn("row_num", row_number().over(w))
awards_salaries3_row.show(5)

+---------+----------+------+-------+-------+
| playerID|   awardID|yearID| salary|row_num|
+---------+----------+------+-------+-------+
|alomaro01|Gold Glove|  1991|1300000|      1|
|alomaro01|Gold Glove|  1992|2983333|      2|
|alomaro01|Gold Glove|  1993|4933333|      3|
|alomaro01|Gold Glove|  1994|5433334|      4|
|alomaro01|Gold Glove|  1995|5500000|      5|
+---------+----------+------+-------+-------+
only showing top 5 rows



In [119]:
result = []
for i in range(1, awards_salaries3_row.count() - 2):
    temp = awards_salaries3_row.filter(col("row_num").between(i, i+2))
    cnt = temp.groupBy('playerID', 'awardID').avg('salary').count()
    if cnt == 1:
        view = temp.collect()
        if view[0][2] + 1 == view[1][2] and view[1][2] + 1 == view[2][2]:
            result.append(temp.groupBy('playerID', 'awardID').avg('salary').collect()[0])

In [124]:
result_df = spark.createDataFrame(result)
result_df.show()
result_df.count()

+---------+--------------+------------------+
| playerID|       awardID|       avg(salary)|
+---------+--------------+------------------+
|alomaro01|    Gold Glove|         3072222.0|
|alomaro01|    Gold Glove|         4450000.0|
|alomaro01|    Gold Glove|         5288889.0|
|alomaro01|    Gold Glove| 5070904.333333333|
|alomaro01|    Gold Glove| 6863464.333333333|
|alomaro01|    Gold Glove| 7332207.333333333|
|alomaro01|  TSN All-Star| 6863464.333333333|
|belleal01|Silver Slugger|2983333.3333333335|
|belleal01|Silver Slugger|         4325000.0|
|belleal01|  TSN All-Star|2983333.3333333335|
|belleal01|  TSN All-Star|         4325000.0|
| bellge02|Silver Slugger|          745000.0|
|beltrca01|    Gold Glove|       1.5255222E7|
|biggicr01|    Gold Glove|3433333.3333333335|
|biggicr01|    Gold Glove|         4360000.0|
|boggswa01|Silver Slugger|1558333.3333333333|
|boggswa01|Silver Slugger|         1725000.0|
|boggswa01|  TSN All-Star|1341666.6666666667|
|boggswa01|  TSN All-Star|1558333.

295

문제 2.

NL리그에서 가장 많이 이긴 팀을 찾으시오.

In [156]:
teams = spark.read.options(header=True).csv('../../data/input4/Teams.csv')
teams = teams.select(teams.teamID, teams.lgID, teams.W)
teams.show(5)

+------+----+---+
|teamID|lgID|  W|
+------+----+---+
|   BS1|  NA| 20|
|   CH1|  NA| 19|
|   CL1|  NA| 10|
|   FW1|  NA|  7|
|   NY2|  NA| 16|
+------+----+---+
only showing top 5 rows



In [157]:
teams.printSchema()

root
 |-- teamID: string (nullable = true)
 |-- lgID: string (nullable = true)
 |-- W: string (nullable = true)



In [158]:
teams = teams.select(
    teams.teamID,
    teams.lgID,
    teams.W.cast('int')
)
teams.show(5)

+------+----+---+
|teamID|lgID|  W|
+------+----+---+
|   BS1|  NA| 20|
|   CH1|  NA| 19|
|   CL1|  NA| 10|
|   FW1|  NA|  7|
|   NY2|  NA| 16|
+------+----+---+
only showing top 5 rows



In [165]:
teams.printSchema()

root
 |-- teamID: string (nullable = true)
 |-- lgID: string (nullable = true)
 |-- W: integer (nullable = true)



In [167]:
teams_sum = teams.where('lgID="NL"').groupBy('teamID', 'lgID').sum('W').withColumnRenamed('sum(W)', 'sum_W')
teams_sum.show(5)

+------+----+-----+
|teamID|lgID|sum_W|
+------+----+-----+
|   BSN|  NL| 5118|
|   BLN|  NL|  644|
|   ML1|  NL| 1146|
|   ML2|  NL|   15|
|   CN1|  NL|  125|
+------+----+-----+
only showing top 5 rows



In [176]:
from pyspark.sql.functions import desc
teams_sum.selectExpr('teamID', 'sum_W').sort(desc('sum_W')).show(1)

+------+-----+
|teamID|sum_W|
+------+-----+
|   CHN|10372|
+------+-----+
only showing top 1 row

